In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split as surprise_train_test_split
from surprise import accuracy



In [2]:
df = pd.read_csv('modified_remedy_data.csv')
print(df)

    Skin_type Sensitivity Blood_group Hyperpigmentation_type  \
0        Oily         Yes          A+            Sun_induced   
1        Oily         Yes          A-            Sun_induced   
2        Oily         Yes          B+            Sun_induced   
3        Oily         Yes          B-            Sun_induced   
4        Oily         Yes          O+            Sun_induced   
..        ...         ...         ...                    ...   
166    Normal          No          B-              Age_spots   
167    Normal          No          O+              Age_spots   
168    Normal          No          O-              Age_spots   
169    Normal          No         AB+              Age_spots   
170    Normal          No         AB-              Age_spots   

                                                Remedy  Remedy_ID  
0    Apple cider vinegar and water toner: Mix 1 par...          1  
1    Strawberry and yogurt mask: Blend strawberries...          2  
2    Avocado and honey mask

In [3]:
df['user_id'] = df[['Skin_type', 'Sensitivity', 'Blood_group', 'Hyperpigmentation_type']].apply(lambda x: '_'.join(x), axis=1)
np.random.seed(42)
df['rating'] = np.random.randint(1, 6, df.shape[0])
print(df)

    Skin_type Sensitivity Blood_group Hyperpigmentation_type  \
0        Oily         Yes          A+            Sun_induced   
1        Oily         Yes          A-            Sun_induced   
2        Oily         Yes          B+            Sun_induced   
3        Oily         Yes          B-            Sun_induced   
4        Oily         Yes          O+            Sun_induced   
..        ...         ...         ...                    ...   
166    Normal          No          B-              Age_spots   
167    Normal          No          O+              Age_spots   
168    Normal          No          O-              Age_spots   
169    Normal          No         AB+              Age_spots   
170    Normal          No         AB-              Age_spots   

                                                Remedy  Remedy_ID  \
0    Apple cider vinegar and water toner: Mix 1 par...          1   
1    Strawberry and yogurt mask: Blend strawberries...          2   
2    Avocado and honey m

In [4]:
reader = Reader(rating_scale=(1, 5)) 
print (reader)
data = Dataset.load_from_df(df[['user_id', 'Remedy_ID', 'rating']], reader)
print(data)


In [5]:
trainset, testset = surprise_train_test_split(data, test_size=0.25)

In [6]:

# A) User-based Collaborative Filtering (UBCF) using KNNBasic
sim_options_user_based = {
    'name': 'cosine',  # Cosine similarity
    'user_based': True  # True for user-based filtering
}


In [7]:
algo_user_based = KNNBasic(sim_options=sim_options_user_based)
algo_user_based.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [8]:
predictions_user_based = algo_user_based.test(testset)
rmse_user_based = accuracy.rmse(predictions_user_based, verbose=False)
# Output the RMSE for UBCF
print(f"User-based CF RMSE: {rmse_user_based}")


User-based CF RMSE: 1.4352958605787944


In [9]:
def predict_remedy_ratings(skin_type, sensitivity, blood_group, hyperpigmentation_type):
    user_id = f"{skin_type}_{sensitivity}_{blood_group}_{hyperpigmentation_type}"
    predictions = []
    remedy_ids = df['Remedy_ID'].unique()
    for remedy_id in remedy_ids:
        pred = algo_user_based.predict(user_id, remedy_id)
        predictions.append((remedy_id, pred.est))
    predictions.sort(key=lambda x: x[1], reverse=True)
    return predictions[0] if predictions else None


In [11]:
skin_type = input("Enter your skin type: ")
sensitivity = input("Enter your sensitivity level: ")
blood_group = input("Enter your blood group: ")
hyperpigmentation_type = input("Enter your hyperpigmentation type: ")
best_remedy = predict_remedy_ratings(skin_type, sensitivity, blood_group, hyperpigmentation_type)
print(best_remedy)

if best_remedy:
    remedy_id, predicted_rating = best_remedy
    remedy_details = df[df['Remedy_ID'] == remedy_id].iloc[0]  # Get details of the recommended remedy
    
    # Print the remedy details along with predicted rating
    print(f"Recommended remedy: {remedy_details['Remedy']} (ID: {remedy_id})")
    print(f"Predicted rating: {predicted_rating:.2f}")
else:
    print("No predictions could be made.")

Enter your skin type:  oily
Enter your sensitivity level:  yes
Enter your blood group:  A+
Enter your hyperpigmentation type:  Hormal, sun_induced 


(1, 2.9140625)
Recommended remedy: Apple cider vinegar and water toner: Mix 1 part apple cider vinegar with 3 parts water. Apply after cleansing. 
Routine: Cleanse, tone, exfoliate 1-2x/week, apply the toner daily, moisturize. (ID: 1)
Predicted rating: 2.91
